<a href="https://colab.research.google.com/github/nomi181472/RL_On_Lunar_lander/blob/main/Hybrid/Hybrid_RL%2BGenetic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!sudo apt-get install swig build-essential  python3-dev xvfb

In [ ]:
!pip install gym gym[box2d] pytorch-lightning pyvirtualdisplay deap

In [3]:

from pyvirtualdisplay import Display
Display(visible=0,size=(1024, 768)).start()

In [4]:
import copy
import gym
import torch
import numpy as np
import torch.nn.functional as F
from collections import deque, namedtuple
from IPython.display import HTML
from base64 import b64encode
import random

In [ ]:
from torch import Tensor, nn
from pytorch_lightning.callbacks import EarlyStopping
from torch.utils.data import DataLoader
from torch.utils.data.dataset import IterableDataset
from torch.optim import AdamW
from pytorch_lightning import Trainer, LightningModule

In [6]:
from gym.wrappers import RecordVideo, RecordEpisodeStatistics,TimeLimit

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
device='cuda:0' if torch.cuda.is_available() else 'cpu'
num_gpus=torch.cuda.device_count()
print(num_gpus)
print(device)

In [8]:
def display_video(episode=0):

  video_file=open(f'/content/videos/rl-video-episode-{episode}.mp4',"r+b").read()
  video_url=f"data:video/mp4;base64,{b64encode(video_file).decode()}"
  return HTML(f"<video width=600 controls><source src='{video_url}'/></video>")



In [9]:
#neural network
class DQN(nn.Module):
  def __init__(self, hidden_state,states,actions):
    super().__init__()
    self.net=nn.Sequential(
        nn.Linear(states,hidden_state,),
        nn.ReLU(),
        nn.Linear(hidden_state,hidden_state),
        nn.ReLU(),
        nn.Linear(hidden_state,actions)

    )
  def forward(self,x):
    return self.net(x.float())



In [10]:
#policy that our agent will follow
def epsilon_greedy(state,env,net,epsilon=0.0):
  action=-1
  if np.random.random()<epsilon:
    action=env.action_space.sample()
  else:
    state=torch.tensor([np.array(state)]).to(device)
    q_values=net(state)
    max_value,action=torch.max(q_values,dim=1)
    action=int(action.item())
  return action
    #[[s1],[s2]] we are getting highest action for that  states





In [11]:
#replay memory buffer
class ReplayBuffer:
  def __init__(self,capacity):
    self.buffer=deque(maxlen=capacity)
  def __len__(self,):
    return len(self.buffer)
  def append(self, experience):
    self.buffer.append(experience)
  def sample(self, batch_size):
    return random.sample(self.buffer,batch_size)





In [12]:
#dataset
class RLDataset(IterableDataset):
  def __init__(self,buffer,sample_size=2000):
    self.buffer=buffer
    self.sample_size=sample_size
  def __iter__(self):
    for experience in self.buffer.sample(self.sample_size):
      yield experience




In [13]:
#creating environment
def create_environment(name,):
  env=gym.make(name)
  env=TimeLimit(env,max_episode_steps=400)
  env=RecordVideo(env,video_folder="./videos",episode_trigger=lambda x:x% 50==0)
  env=RecordEpisodeStatistics(env) #history of success and failure
  return env

In [ ]:

env=create_environment("LunarLander-v2")

for episode in range(10):
  env.reset();
  done=False
  while not done:
    action=env.action_space.sample()
    _,_,done,_=env.step(action)


In [ ]:
env.action_space.sample()

In [20]:
import heapq
class GA:
    def __init__(self, hidden_layer, obj_size, n_actions, population_size=100, cross_over_rate=0.1,
                 mutate_rate=0.1,

                 ):
        self.obj_size = obj_size
        self.hidden_layer = hidden_layer
        self.n_actions = n_actions

        self.population_size = population_size

        self.cross_over_rate = cross_over_rate
        self.mutate_rate = mutate_rate
        self.populations = []
        self.population_performances = []
        self.generate_initial_population()

    def get_flatten_list(self, index):
        flattened_weights = []
        model = self.populations[index]
        for layer in model.net.children():
            if isinstance(layer, torch.nn.Linear):
                weights = layer.weight.view(-1).tolist()  # Flatten the weights and convert to a list
                flattened_weights.extend(weights)
        return flattened_weights

    def crossover(self, parent1, parent2):
        child = []
        for i in range(len(parent1)):
            if i % 2 == 0:
                child.append(parent2[i])
            else:
                child.append(parent1[i])
        return child

    def mutate(self, child):
        for i in range(int(len(child) * self.mutate_rate)):
            index = np.random.randint(0, len(child) - 1)
            child[index] = np.random.normal()
        return child

    @torch.no_grad()
    def mating(self, ):
        # Get the top 10 maximum numbers with indices using a heap
        selection = 76
        top_max = heapq.nlargest(selection, enumerate(self.population_performances), key=lambda x: x[1])
        top_indices = [index for index, value in top_max]
        for i in range(int(selection / 2)):
            # Todo selection
            first_parent_index = random.choice(top_indices)
            top_indices.remove(first_parent_index)
            second_parent_index = random.choice(top_indices)
            top_indices.remove(second_parent_index)
            # Todo flatten
            first_parent = self.get_flatten_list(first_parent_index)
            second_parent = self.get_flatten_list(second_parent_index)
            # Todo Crossover
            child = self.crossover(first_parent, second_parent)
            # Todo Mutation
            child = self.mutate(child)
            # Todo child expected score
            child_expected_score = (self.population_performances[first_parent_index] + self.population_performances[
                second_parent_index]) / 2
            # Todo replace worst solution
            worst_solution = min(self.population_performances)
            worst_solution_index = self.population_performances.index(worst_solution)
            self.population_performances[worst_solution_index] = child_expected_score
            model = self.populations[worst_solution_index]
            # Todo back to list
            index = 0
            for layer in model.net.children():
                if isinstance(layer, torch.nn.Linear):
                    # Calculate the number of weights in the current layer
                    num_weights = layer.weight.numel()
                    # Extract the weights from the combined list and reshape them
                    new_weights = torch.tensor(child[index:index + num_weights]).view(layer.weight.size())
                    # Assign the modified weights back to the layer
                    layer.weight.data = new_weights
                    # Move to the next set of weights in the combined list
                    index += num_weights

            self.populations[worst_solution_index]=model

    def generate_initial_population(self):
        for i in range(self.population_size):
            self.populations.append(DQN(self.hidden_layer, self.obj_size, self.n_actions))  # qtable)
            # model = DQN(self.hidden_layer, self.obj_size, self.n_actions)
            # flattened_weights = []
            # # for layer in model.net.children():
            # #     if isinstance(layer, torch.nn.Linear):
            # #         weights = layer.weight.view(-1).tolist()  # Flatten the weights and convert to a list
            # #         flattened_weights.extend(weights)
            # # self.populations.append(flattened_weights)
            # for idx, layer in enumerate(model.net.children()):
            #     if isinstance(layer, torch.nn.Linear):
            #         input_size = layer.in_features
            #         output_size = layer.out_features
            #         print(f"Layer {idx + 1}: Input Size: {input_size}, Output Size: {output_size}")



class DeepQLearning(LightningModule):
  #initialize
  def __init__(self,
               env_name, policy=epsilon_greedy,capacity=100_000,
               batch_size=256, lr=1e-3,hidden_layer=128,gamma=0.99,
               loss_fn=F.smooth_l1_loss,optim=AdamW,eps_start=0,
               eps_end=0.15,eps_last_episode=100,sample_per_epoch=10_000,sync_rate=100,
               max_iters=700
               ):
    super().__init__()
    self.env=create_environment(env_name)
    obj_size=env.observation_space.shape[0]
    n_actions=env.action_space.n
    self.GA = GA(hidden_layer, obj_size, n_actions)
    self.q_net = self.get_max_performer()
    #self.q_net=DQN(hidden_layer,obj_size,n_actions)
    self.target_q_net=copy.deepcopy(self.q_net)
    self.policy=policy
    self.max_reward=0
    self.max_iters=max_iters
    self.buffer=ReplayBuffer(capacity=capacity)
    self.save_hyperparameters()
    while len(self.buffer)<self.hparams.sample_per_epoch:
      print(f"{len(self.buffer)} sampes in experieces buffer filling.... ")
      self.play_episode(epsilon=self.hparams.eps_start)




  @torch.no_grad()
  def get_max_performer(self):
      index = 0
      highest_reward = float('-inf')
      for idx, solution in enumerate(self.GA.populations):
          solution.train()
          state = self.env.reset()
          state = torch.tensor(state, dtype=torch.float32)
          total_reward = 0
          for i in range(700):
              # Compute prediction error

              pred = solution(state)
              action = torch.argmax(pred).item()

              next_state, reward, done, info = self.env.step(action)
              next_state = torch.tensor(next_state, dtype=torch.float32)
              total_reward = total_reward + reward
              state = next_state
              if done:
                  break
          if total_reward > highest_reward:
              highest_reward = total_reward
              index = idx
          self.GA.population_performances.append(total_reward)
      return self.GA.populations[index]

  @torch.no_grad()
  def play_episode(self,policy=None,epsilon=0.):
    #print("play_episode")
    state=self.env.reset()
    done =False
    action=2
    while not done:

      if policy:
        action=policy(state,self.env,self.q_net,epsilon=epsilon)
        #print(f"policy available, action: {action} {policy}")
      else:
        action=self.env.action_space.sample()
        #print(f"policy available, action: {action}")
      next_state,reward,done,info=self.env.step(action)
      exp=(state,action,reward,done,next_state)
      self.buffer.append(exp)
      state=next_state
    #print("play_episode-done")


  #forward
  def forward(self,x):
    #print("forward")
    return self.q_net(x);

  #configure Optimizer
  def configure_optimizers(self):
    #print("configure_optimizers")
    q_net_optimizer=self.hparams.optim(self.q_net.parameters(),lr=self.hparams.lr)
    #print("configure_optimizers-done")
    return [q_net_optimizer]
  def train_dataloader(self):
    #print("train_dataloader")
    dataset=RLDataset(self.buffer,self.hparams.sample_per_epoch);
    dataloader=DataLoader(
        dataset=dataset,
        batch_size=self.hparams.batch_size
    )
    #print("train_dataloader-done")
    return dataloader
  #training steps
  def training_step(self,batch,batch_ids):
    #print("training_step")

    states,actions,rewards,dones,next_states=batch
    actions=actions.unsqueeze(1)
    rewards=rewards.unsqueeze(1)
    dones=dones.unsqueeze(1)

    state_action_values=self.q_net(states).gather(1,actions)

    next_action_values,_=self.target_q_net(next_states).max(dim=1,keepdim=True)
    next_action_values[dones]=0.0

    expected_state_action_values=rewards +self.hparams.gamma * next_action_values

    loss= self.hparams.loss_fn(state_action_values,expected_state_action_values)
    self.log("episode/Q-error",loss)
    #print("training_step-done")
    return loss
  #training epoch end
  def on_train_epoch_end(self):
    #print("on_train_epoch_end")

    epsilon=max(self.hparams.eps_end,self.hparams.eps_start-self.current_epoch/self.hparams.eps_last_episode)
    self.play_episode(policy=self.policy,epsilon=epsilon)
    self.log("episode/return",self.env.return_queue[-1])
    if self.env.return_queue[-1]>self.max_reward:
      self.max_reward=self.env.return_queue[-1]
    self.GA.mating()
    print(f"Epoch:{self.current_epoch}: Max Reward:{self.max_reward}")
    if self.current_epoch %self.hparams.sync_rate==0:
      self.target_q_net.load_state_dict(self.q_net.state_dict())
    #print("on_train_epoch_end-done")


In [ ]:
!rm -r /content/lightning_logs/
!rm -r /content/videos/
%load_ext tensorboard
%tensorboard --logdir /content/lightning_logs/

In [ ]:
algo=DeepQLearning("LunarLander-v2")


In [ ]:
trainer=Trainer(accelerator="cuda",max_epochs=10_000,
                callbacks=[EarlyStopping(monitor="episode/return",mode="max",patience=500)])
trainer.fit(algo)

In [ ]:
display_video(800)